# SB3 Policy



在Stable Baselines3 (SB3)中，策略网络（Policy Networks）是用于处理来自环境的观察值并基于这些观察值输出决策或估计值的关键组成部分。具体到SB3，这些建模观察值与决策（连续或离散动作空间）之间映射的网络不仅仅局限于"智能体采取动作的决策"这一层面，而是扩展到所有参与学习流程的网络。例如，对于Actor-Critic方法如PPO，"Policy"会同时包含参与估算价值函数（Critic）的网络。

### 主要组件

#### 特征提取器
- 负责从环境的（可能是）高维观察变量中提取关键的表示向量（即，将观察向量转换为特征向量）。例如，从图像中，CNN特征提取器可能会学会发掘有助于智能体行动的关键视觉元素。
- 特征提取通常在Actor和Critic之间共享（如果适用），以减少计算开销。

#### 网络架构（Net Architecture）
- 基于以上获得的特征表示，映射至智能体的**动作**或者价值**评价**的网络架构，这主要通过一个或多个全连接网络实现。
- 其结构通过`net_arch`参数加以控制。

### 策略（Policy）类型

- **CnnPolicies**：针对基于图像的环境设计的网络策略。
- **MlpPolicies**：针对传统的向量观察而设计的多层全连接前馈网络。
- **MultiInputPolicies**：针对那些拥有不同形态输入，例如字典输入，即一个观察值中既包含图像又包含其他类型向量数据的环境。

### 动作空间处理
- 在A2C和PPO中，当面对连续动作空间时，连续动作在训练和测试期间会被裁剪，以免产生越界错误。
- 相比之下，SAC、DDPG和TD3通过使用`tanh()`激活函数等转换机制处理连续动作空间的界限问题。

### 注意
所有观察在送入特征提取器前都会被先行处理。例如，基于图像的观察点将被标准化，而离散的观察点会被转换为独热向量。如果观察是一个简单的线性观察向量，那么这个步骤可能只不过是一个Flatten层。

SB3中的*Policy*，特别是Actor-Critic方法中，它既指定了价值估算网络（Critic）的设计，也制定了智能体动作输出策略的构造，还可能包含目标网络等其他元素。因此，"Policy"在SB3的术语中使用范围更广，不只局限于通常的动作决策概念。 

![在这里插入图片](https://stable-baselines3.readthedocs.io/en/master/_images/net_arch.png) 

图解说明了网络架构。如图，Actor和Critic通常有独立的网络架构，每个架构中包括了特征提取和基于这些特征的决策制定模块。对目标网络适用的场景中，也体现了这样的结构。 以上解释了SB3中`Policy`的设计哲学和关键组件。它通过灵活的网络架构适应了不同环境所需的复杂性，有着广泛的适用范围。通过定制这些建筑块，研究人员能够针对其独特环境的需求开发有效的学习模型。

![第二个图](https://stable-baselines3.readthedocs.io/en/master/_images/sb3_policy.png)

## Default Network Architecture

当使用Stable Baselines3 (SB3)时，默认的网络架构因算法和观察空间的不同而有所差异。为了了解网络架构的详细配置，可以通过打印`model.policy`命令来直观查看（请参考SB3的问题讨论#329）。

### 1D观察空间

对于1D（一维）观察空间（典型的情况是一个连续的、扁平化的向量表示形式），SB3默认采用以下全连接网络架构：

- **PPO/A2C/DQN**：采用两层全连接网络，每层64个单元。
- **SAC**：采用两层全连接网络，但每层扩增至256个单元。
- **TD3/DDPG**：采用两层全连接网络，第一层400个单元，第二层300个单元，这一设计直接来源于原始TD3文章的建议。

### 图像观察空间

对图像观察数据，SB3采用著名的“Nature CNN”架构用于图像的特征抽取：

- 对A2C和PPO（在策略算法）而言，Actor和Critic之间共享CNN以降低计算量。
- 对TD3、DDPG和SAC这些离策略算法来说，Actor和Critic会有独立的CNN特征提取模块，这配置被实验验证能获得更优的性能。

### 混合观察空间（字典观察空间）

对字典型（即混合型）观察数据，SB3会结合上述两种不同观察情况的网络架构：

- 对图像成分，使用“Nature CNN”进行特征抽取；
- 同时为了处理可能伴随的其他数值向量型观察数据，再接一两层的全连接层进行后续处理，典型的后接网络在这一场景下可能相对更小（为了平衡整体的参数规模和处理图像部分所抽取的特征的表示能力）。

请留意，在不同场景中这些建议的网络架构都是灵活和可调整的。通过适当的调参（如调整神经网络的宽度、深度等），研究者可以针对自己的任务需求对网络架构进行微调，从而获得最优的表现。此外，SB3还允许研究者根据任务的特殊需求自定义神经网络结构和特征提取器。这为处理复杂观察空间提供了巨大的灵活性和潜力，有益于深入挖掘观察数据中潜藏的、对任务决策帮助最大的信息。

## 定制网络架构 Custom Network Architecture

自定义网络架构是调整强化学习算法性能的关键方法之一。通过使用`policy_kwargs`参数在创建模型时传递自定义参数，可以轻松实现网络架构的个性化定制。在本例中，我们将探讨如何在使用PPO算法时，针对`CartPole-v1`环境自定义actor（策略网络）和critic（价值网络）的架构。

### 自定义网络架构示例

以下代码段展示了如何在使用PPO算法建模时自定义网络架构：

```python
import gym
import torch as th
from stable_baselines3 import PPO

# 自定义actor（pi）和critic（vf）网络
# 这两个网络均有2个隐藏层，每层32个单元，并使用ReLU作为激活函数
# 注意：在pi和vf网络的顶部分别会自动添加一个额外的线性层，
# 以确保输出尺寸正确，以及动作空间的激活函数（如：离散动作使用Softmax）
policy_kwargs = dict(activation_fn=th.nn.ReLU,
                     net_arch=dict(pi=[32, 32], vf=[32, 32]))

# 创建PPO模型实例
model = PPO("MlpPolicy", "CartPole-v1", policy_kwargs=policy_kwargs, verbose=1)

# 训练模型
model.learn(total_timesteps=20_000)

# 保存模型
model.save("ppo_cartpole")

# 删除当前模型实例
del model

# 加载保存的模型，policy_kwargs参数会自动加载
env = gym.make("CartPole-v1")
model = PPO.load("ppo_cartpole", env=env)
```

### 代码细节说明

- 首先，我们通过设置`policy_kwargs`字典来自定义我们的网络架构，包括指定ReLU作为激活函数，以及为actor（`pi`）和critic（`vf`）分别配置两个拥有32个神经单元的隐藏层。

- 随后，我们创建了针对`CartPole-v1`环境的PPO模型实例，并将自定义的网络架构通过`policy_kwargs`参数传递给模型。

- 之后，我们使用`learn`方法训练PPO模型，累计20,000个时间步。

- 在模型训练完成后，我们将模型保存到磁盘上。

- 最后，我们通过调用`PPO.load`方法重新加载保存的模型，并将环境实例一并作为参数提供。请注意，虽然这里没有显式地传递`policy_kwargs`参数，但之前的自定义网络架构已被妥善保存，可以自动加载。

这个自定义网络架构的例程非常有助于了解如何在实践中对模型的表示能力进行调整，以改进针对具体任务的性能。通过灵活的定义`policy_kwargs`参数，可以实现多样化的模型结构设计，满足不同研究和开发目标的需求。

我们将通过创建一个针对连续控制任务（如`Pendulum-v0`环境）的自定义网络架构的示例，来深入阐述如何在PPO算法中使用`policy_kwargs`参数进行个性化的网络设计。这一任务将向我们展示在解决需要精密连续控制动作的环境时，如何使用深度学习算法对模型策略网络进行调整。

### 示例代码

```python
import gym
from stable_baselines3 import PPO
from torch import nn

# 定义自定义的Actor和Critic网络架构
# 这对网络在处理具体任务时，往往需要基于任务特性进行设计调整
policy_kwargs = dict(
    activation_fn=nn.Tanh,  # 激活函数采用Tanh
    net_arch=[128, 128, dict(pi=[64, 32], vf=[64, 32])]  # 自定义神经网络架构
)

# 创建环境实例
env = gym.make("Pendulum-v0")

# 创建模型实例，这里我们使用PPO算法
model = PPO("MlpPolicy", env, verbose=1, policy_kwargs=policy_kwargs)

# 训练模型
model.learn(total_timesteps=10000)

# 保存模型
model.save("ppo_pendulum_custom")

# 如需之后使用，加载保存的模型
del model  # 删除当前模型实例
model = PPO.load("ppo_pendulum_custom")
```

### 代码详细说明

1. **自定义Actor和Critic网络架构**：
    - 通过`policy_kwargs`参数，我们设置网络架构来个性化Actor和Critic的网络。
    - 具体而言，整个网络首先通过两个全连接层，每层128个单元，并应用`Tanh`激活函数。
    - 然后针对Actor(`pi`)和Critic(`vf`)的网络在此基础之上分别应用一个额外的两层全连接网络，分别具有64和32个单元。

2. **环境和模型**：
    - 我们使用OpenAI Gym的`Pendulum-v0`环境，这是一个连续动作空间的挑战环境，经常用于测试连续动作空间的强化学习算法。
    - 我们采用PPO算法来处理这个连续的动作控制问题，并传入自定义网络架构通过`policy_kwargs`参数。

3. **模型训练与保存**：
    - 模型在环境上进行了10,000个时间步的学习。
    - 学习完成后，我们将学习到的智能体策略保存到文件中，以便将来进一步的使用或分析。

4. **加载模型**：
    - 演示了如何在删除当前实例后重新加载保存的智能体模型。

通过自定义Actor和Critic网络架构，本示例不仅阐述了在PPO算法中针对给定任务个性化调整网络配置的方法，同时也表明了这种灵活的网络自定义能力是对深度增强学习研究和实际应用中针对特定任务特性做进一步探索和尝试的基础。这为在多样化任务环境中部署和研究不同复杂性和性能的智能体策略提供了广泛的可能性。

## 自定义特征提取器 Custom Feature Extractor

如果你打算针对处理图像观察的环境自定义一个特征提取器，比如自定义的CNN，可以通过从`BaseFeaturesExtractor`派生一个新的类来实现。下面是针对这个设想的步骤和说明。

### 自定义CNN特征提取器

1. **定义你的CNN特征提取类**：首先，你需要定义一个自定义的CNN特征提取器类，该类从`BaseFeaturesExtractor`派生。这意味着它需要实现`__init__`方法和`forward`方法。`__init__`中，你可以定义CNN的架构；而在`forward`方法中，你将指定如何通过这个CNN处理观察数据来提取有用的表示。

2. **初始化CNN模块**：在这个自定义CNN类的初始化方法中，你可以根据观察空间的形状设置CNN的不同层次结构。首先确定输入图像的通道数，接着你可以像通常使用PyTorch那样搭建CNN层次。

3. **前向计算**：`forward`方法负责获取观察值张量并通过CNN进行处理，最后返回一个表示学习到的观察表示的张量。

### 应用自定义CNN特征提取器

1. **设置`policy_kwargs`参数**：一旦你定义了自定义的CNN特征提取器，接下来就是在创建PPO模型时通过`policy_kwargs`参数把这个自定义的CNN类告诉模型。确保`features_extractor_class`关键词参数设置为你的自定义CNN类。

2. **调整特征表示尺寸**：通过`features_extractor_kwargs`关键词参数字典的`features_dim`键设置期望的表示特征尺寸。确保这与自定义CNN最后输出的表示形状吻合。

3. **创建PPO模型实例**：接下来，创建PPO算法的实例时，你需要为`policy`参数选择"CnnPolicy"，并传递`policy_kwargs`。

### 完整代码示例

```python
import torch as th
import torch.nn as nn
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

class CustomCNN(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.spaces.Box, features_dim: int = 256):
        super().__init__(observation_space, features_dim)
        n_input_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
        )
        # Compute shape by doing one forward pass
        with th.no_grad():
            n_flatten = self.cnn(th.as_tensor(observation_space.sample()[None]).float()).shape[1]
        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

    def forward(self, observations: th.Tensor) -> th.Tensor:
        return self.linear(self.cnn(observations))

policy_kwargs = dict(
    features_extractor_class=CustomCNN,
    features_extractor_kwargs=dict(features_dim=128),
)

model = PPO("CnnPolicy", "BreakoutNoFrameskip-v4", policy_kwargs=policy_kwargs, verbose=1)
model.learn(1000)
```

本例通过定义一个自定义CNN类`CustomCNN`来充分说明了如何使用PPO模型为图像观察空间预设个性化CNN结构。此举可增加灵活性，使得针对观察数据的复杂度或领域知识的掌握程度有针对性地调整表示提取部分。通过在`policy_kwargs`中明确传递这个特征提取器类以及期望的表示尺寸，便可以轻松将自定义

的表示能力嵌入到增强学习流程中，拓宽了研究人员和开发人员解决任务的工具集。

本例旨在为处理文本输入数据的环境开发一个自定义的循环神经网络（RNN）作为特征提取器。我们将通过从`BaseFeaturesExtractor`派生自定义RNN类，并将其应用于PPO模型中，以处理某种形式的序列化文本观测数据。

### 第一步：定义自定义RNN特征提取器

首先，定义自定义RNN类，从`BaseFeaturesExtractor`派生，并实现必需的`__init__`方法和`forward`方法。此RNN旨在将来自环境观察的嵌入式文本序列映射到连续的表示矢量。

```python
import torch as th
import torch.nn as nn
from gymnasium import spaces
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

class CustomRNN(BaseFeaturesExtractor):
    def __init__(self, observation_space: spaces.Box, features_dim: int = 256):
        super(CustomRNN, self).__init__(observation_space, features_dim)
        
        # 假设observation_space是嵌入向量的序列，其中序列长度为10，每个嵌入的大小为50
        self.rnn = nn.GRU(input_size=50, hidden_size=features_dim, num_layers=1, batch_first=True)

    def forward(self, observations: th.Tensor) -> th.Tensor:
        # 假设observations的形状为[batch_size, seq_len, embedding_dim]
        _, hn = self.rnn(observations)
        # hn的形状为[num_layers, batch_size, hidden_size]，这里我们只关注最后一层的输出
        return hn.squeeze(0)
```

### 第二步：将自定义RNN应用于PPO模型

现在我们拥有了一个针对嵌入式序列观测数据设计的自定义RNN特征提取器，下一步是将其整合到PPO策略网络中：

```python
from stable_baselines3 import PPO
from gymnasium import spaces

# 假设观测空间是10个连续的嵌入文本表示，嵌入尺寸为50
observation_space = spaces.Box(low=-float('inf'), high=float('inf'), shape=(10, 50), dtype=th.float32)

# 自定义PPO策略网络的设置
policy_kwargs = dict(
    features_extractor_class=CustomRNN,
    features_extractor_kwargs=dict(features_dim=128),  # 特征表示的目标尺寸
)

# 创建环境（为了演示方便，这里没有指定具体环境，应该替换为实际环境）
env = 'YourEnvHere'

# 创建PPO模型实例
model = PPO('MlpPolicy', env, policy_kwargs=policy_kwargs, verbose=1)

# 进行学习
# model.learn(total_timesteps=20000)
```

### 代码详细说明

- **自定义RNN特征提取器**：`CustomRNN`基于`nn.GRU`实现了简单的循环网络结构，其设计针对序列化嵌入表示的观察数据。通过将序列处理转换为定长的连续表示，适合随后的决策过程。

- **集成RNN到PPO**：通过在`policy_kwargs`中明确地提及自定义RNN类，并赋予希望的表示能力参数，可以轻松将复杂观测转换融入现代的深度增强学习方法中。

- **灵活的表示能力**：设计中的关键因素在于，自定义RNN是作为观察数据特征抽取阶段的延伸，解放了对复杂和序列化观测空间处理的灵活能力。

### 注意

- 这个例证用了一个伪造的环境名称`'YourEnvHere'`和嵌入式观测的结构，实际使用时，你需要替换

为针对你的实际应用场景开发的环境。
- `model.learn(total_timesteps=20000)`代码行在本例子中被注释掉了，但在实践应用中应取消注释以开始模型的学习。

通过这一机制，可以针对不同形态和不平凡的观察输入提供细致和有针对性的表示学习策略，增进了处理复杂环境任务的灵活性和潜能。这种个性化订制增强了深度增强学习模型针对观察数据表示的理解深度，为开发出适应性更强、性能更优的增强学习代理铺平了道路。 

## 多个输入和字典观察 Multiple Inputs and Dictionary Observations

在处理复杂环境时，观察数据可能具有不同的形态，如图像、向量或两者的混合。针对这样的情况，Stable Baselines3通过`Dict`观察空间和`MultiInputPolicy`的设计，以及结合自定义的`BaseFeaturesExtractor`派生类，为开发人员提供灵活地处理不同输入类型并将其转换为单个表示向量的能力。

### 自定义组合式特征提取器

以下是创建自定义组合式特征提取器的完整示例，旨在处理字典观察值中既有图像也有向量数据的环境：

```python
import gym
import torch as th
from torch import nn
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

class CustomCombinedExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.spaces.Dict):
        super().__init__(observation_space, features_dim=1)  # 先设定虚拟的features_dim

        extractors = {}
        total_concat_size = 0

        for key, subspace in observation_space.spaces.items():
            if key == "image":
                # 简单的针对图像应用一个4x4的平均池化和扁平化
                # 这里默认图像是单通道
                extractors[key] = nn.Sequential(nn.MaxPool2d(4), nn.Flatten())
                total_concat_size += subspace.shape[1] // 4 * subspace.shape[2] // 4
            elif key == "vector":
                # 简单的全连接网络处理向量数据
                extractors[key] = nn.Linear(subspace.shape[0], 16)
                total_concat_size += 16

        self.extractors = nn.ModuleDict(extractors)
        self._features_dim = total_concat_size  # 根据处理后的数据更新总的输出尺寸

    def forward(self, observations) -> th.Tensor:
        encoded_tensor_list = []
        for key, extractor in self.extractors.items():
            encoded_tensor_list.append(extractor(observations[key]))
        # 在最后的维度上把所有处理后的张量（Tensor）进行并联
        return th.cat(encoded_tensor_list, dim=1)
```

### 应用自定义组合式特征提取器

1. **字典观察空间**：首先，环境应该返回一个字典观察空间，这在很多复杂的任务中较为常见，字典中可以包含不同形式和尺度的观察数据，例如图像加向量数据。

2. **自定义处理不同类型的输入数据**：本例通过针对名为`"image"`的键值对下采样处理图像数据，并通过线性层处理向量（数值）数据名为`"vector"`的键值。

3. **并联不同处理后的表示**：对于提取的表示特征向量，通过拼接操作把所有处理后的表示并联起来，形成一个整体的、表示混合观察输入的单一向量。

4. **灵活地扩展环境表示能力**：上述结构之美在于能将不同模态的观察表示转换为智能体决策可以直接处理的恒定形式，充分提高表示学习能力的灵活性和多样性。

5. **与PPO或其他SB3算法集成**：一经定义了自定义的组合式特征提取器，便可以通过传递`policy_kwargs`参数在模型创建时把它集成到诸如PPO之类的学习算法之中，有效支持不同模态的环境表示学习。

通过灵活自定义观察到的表示转换流程，进而针对具体任务的环境表示学习需求设计最佳的表示抽取模型，显著增强基于深度强化学习算法处理多模态观察输入的能力。



本示例旨在介绍如何为多输入类型环境（即环境观察值为字典形式，字典内既有图像也有其他非图像数值数据）设计自定义的混合类型观察表示提取器。此表示提取器的目的是将环境返回的不同数据形态统一转化为适用于深度学习策略（如PPO、SAC等）的表示形式。

### 第一步：准备环境

我们先设想有一个环境观察字典返回两种关键信息：一是环境当前状态的RGB图像（`'image'`），二是环境的向量描述形式（`'vector'`）。针对此环境类型，我们将设计一个能够处理并组合这些混合类型数据的自定义表示提取器。

### 第二步：自定义表示提取器

```python
import torch as th
from torch import nn
import gymnasium as gym
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

class HybridFeaturesExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.spaces.Dict, features_dim: int = 256):
        super(HybridFeaturesExtractor, self).__init__(observation_space, features_dim)
        
        # 处理图像数据的CNN特征提取器
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2),  # 假设图像数据3个通道，如RGB
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2),
            nn.ReLU(),
            nn.Flatten()
        )
        
        # 处理向量数据的简单线性变换
        vector_input_dim = observation_space.spaces['vector'].shape[0]
        self.vector_net = nn.Sequential(
            nn.Linear(vector_input_dim, 64),
            nn.ReLU()
        )
        
        # 合并的全连接层用以生成最终表示
        self.final_net = nn.Sequential(
            nn.Linear(64 + 32 * 7 * 7, features_dim),  # 假定合并的CNN输出大小为32*7*7
            nn.ReLU()
        )

    def forward(self, observations) -> th.Tensor:
        # 使用CNN处理图像数据
        image_output = self.cnn(observations['image'])
        # 使用全连接网络处理非图像向量数据
        vector_output = self.vector_net(observations['vector'])
        # 合并得到的表示
        merged_output = th.cat((image_output, vector_output), dim=1)
        # 通过最终的全连接层生成所需尺寸的表示
        return self.final_net(merged_output)
```

### 第三步：模型与表示提取器的集成

```python
from stable_baselines3 import SAC
from gymnasium.spaces import Dict, Box, Discrete
import gymnasium as gym

# 模拟环境观察字典和行为字典构建
observation_space = gym.spaces.Dict({
    'image': gym.spaces.Box(low=0, high=255, shape=(3, 84, 84), dtype=th.uint8),
    'vector': gym.spaces.Box(low=-1, high=1, shape=(10,))
})
action_space = gym.spaces.Box(low=-1, high=1, shape=(3,))

# 假设Env
class SimulatedEnv(gym.Env):
    def __init__(self):
        self.observation_space = observation_space
        self.action_space = action_space

# 自定义策略关键参数
policy_kwargs = dict(
    features_extractor_class=HybridFeaturesExtractor,
    features_extractor_kwargs=dict(features_dim=256)
)

# 创建环境实例和SAC模型实例
env = SimulatedEnv()
model = SAC("MultiInputPolicy", env, policy_kwargs=policy_kwargs, verbose=1)

# 模型学习
model.learn(total_timesteps=1000)
```

### 解释

- **自定义表示提取器**：本实例先展示了针对既含图像数据也含附加向量数据的环境观察的复合型自定义表示提取器。通过并联处理这些子

空间，我们设计的`HybridFeaturesExtractor`生成了表征环境观察字典中不同模态数据的表示。

- **表示融合与处理**：图像子观察使用CNN加以处理，而向量数据则通过单独的线性层处理。处理得到的不同表示向量随后在通过全连接层并联融合，而后可被智能体用于后续地决策制定。

- **集成表示提取器**：通过自定义的`HybridFeaturesExtractor`，并通过`policy_kwargs`显式提供以将其加入SAC模型，我们能够针对同时存在多种形式观察输入的环境应用深度增强学习算法。

通过灵活而有针对性的设计和嵌入表示提取器，我们不仅为应对环境多模态观察数据挑战铺平了道路，还大幅提升了智能体决策策略在获取关键环境信息表示能力上的自适应性和鲁棒性。这意味着，通过此方法，复杂环境观察处理与表示生成得以高效整合，有望显著地增强模型实际部署性能。

# 在线策略算法On-Policy Algorithms

本高级示例旨在向您展示如何在对PPO算法使用时，针对Actor（智能体）和Critic（价值函数）应用不同的网络架构。通过自定义ActorCriticPolicy，本例阐述在有着独特体系结构需求的复杂增强学习任务中，对特征抽取器和策略-价值函数网络的深入配置方法。

### 步骤概述

1. **自定义网络定义**：
    - 本例中，我们创建了`CustomNetwork`类，该类分别针对Actor和Critic网络嵌套地实现定制化结构，它们共享来自环境观察值的初始表征，但随后应用不同的后处理路径。

2. **自定义ActorCriticPolicy**：
    - 接下来，通过从`ActorCriticPolicy`派生来定义`CustomActorCriticPolicy`类。这种方式赋予自定义处理特征表示（来自环境观察值）的高级灵活性，同时控制Actor和Critic的决策生成。

3. **MLP提取器的改写**：
    - 在`CustomActorCriticPolicy`中，通过复写`_build_mlp_extractor`方法，可以集成先前定义的`CustomNetwork`作为策略的MLP提取器。

4. **PPO算法实例化与自定义Policy集成**：
    - 最后，我们实例化PPO模型，并传入`CustomActorCriticPolicy`，这提供对智能体表示函数架构的最大控制权。

### 核心代码段详细分析

（完整示例代码请见[官网](https://stable-baselines3.readthedocs.io/en/master/guide/custom_policy.html#on-policy-algorithms)。）

1. **自定义Representation类**

```python
class CustomNetwork(nn.Module):
    def __init__(self, feature_dim: int, last_layer_dim_pi: int = 64, last_layer_dim_vf: int = 64):
        super().__init__()
        self.policy_net = nn.Sequential(nn.Linear(feature_dim, last_layer_dim_pi), nn.ReLU())
        self.value_net = nn.Sequential(nn.Linear(feature_dim, last_layer_dim_vf), nn.ReLU())
    
    def forward(self, features: th.Tensor) -> Tuple[th.Tensor, th.Tensor]:
        return self.forward_actor(features), self.forward_critic(features)
    
    def forward_actor(self, features: th.Tensor) -> th.Tensor:
        return self.policy_net(features)
    
    def forward_critic(self, features: th.Tensor) -> th.Tensor:
        return self.value_net(features)
```

- 这一部分代码定义了一个自定义Representation网络，能够根据智能体从环境中获取到的表示去生成Actor和Critic特有的决策或值函数输出。这通过定义特异性地针对Actor和Critic路径的模块化的顺序容器来实现。

2. **自定义ActorCriticPolicy类**

```python
class CustomActorCriticPolicy(ActorCriticPolicy):
    def __init__(self, observation_space: spaces.Space, actionspace: spaces.Space, lr_schedule: Callable[[float], float], *args, **kwargs):
        kwargs["ortho_init"] = False
        super().__init__(observation_space, action_space, lr_schedule, *args, **kwargs)

    def _build_mlp_extractor(self) -> None:
        self.mlp_extractor = CustomNetwork(self.features_dim)
```

- 这里，我们通过复写`_build_mlp_extractor`函数，集成了上面所定义的自定义Representation网络。通过该函数对PPO算法所采用的底层MLP表示函数进行定制化的修改，以此实现对环境不同观察形式下高维表示的差异化处理。

### 应用自定义ActorCriticPolicy类

```python
model = PPO(CustomActorCriticPolicy, "CartPole-v1", verbose=1)
model.learn(5000)
```

- 这一步骤展示如何使用自定义Policy进行PPO训练。通过这种个性化Policy的方法，研究者可以灵活设定针对Actor和Critic使用差异化的表示能力和决策能力，尤其在面对高度复杂或需求特殊化的环境设定下，这成为实现尖端增强学习表现的有力工具。

通过本例，您可以更深刻理解如何为不断发展的复杂任务在PPO、A2C或TRPO等算法下灵活配置差异化的表示能力和学习决策。这不仅大幅度扩展算法的适用范围，而且推进了个性化和精准化表示习得的边界，满足更广泛增强学习场景的需求。

本例中，我们将开发一个自定义Actor-Critic策略，旨在差异化处理连续控制任务（比如OpenAI Gym中的`Pendulum-v1`环境）的Actor网络和Critic网络。特别地，针对Actor网络，我们希望应用一层较浅的网络架构（较少的隐藏单元），而针对Critic网络，我们打算部署一层较深的网络架构（较多的隐藏单元），以此捕捉环境动态的复杂表示。

### 第一步：自定义Representation类

首先，让我们构造自定义Representation类。和前一个例子不同，这里不会显式定义特征抽取网络和MLP（多层感知器）表示。我们将专注于差异化Actor和Critic的表示。

### 第二步：自定义ActorCritic策略

这里，我们将使用PyTorch的`nn.Module`来界定差异化神经网络架构。Actor将使用较浅的网络结构（较少的参数），而Critic应用较深的网络结构（较多的参数）。

### 第三步：PPO模型实例化和应用自定义表示

最终，实例化PPO算法模型，并指明所针对的环境以及如何通过自定义表示实现差异化网络架构。

### 代码实践

```python
import gym
import torch as th
from torch import nn
from stable_baselines3 import PPO
from stable_baselines3.common.policies import ActorCriticPolicy
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

# 第一步：自定义观察表示类
class CustomObservationExtractor(BaseFeaturesExtractor):
    """
    自定义观察表示，可针对环境的具体观察设计表示提取结构。
    这里简单定义为通用结构。
    """
    def __init__(self, observation_space: gym.spaces.Box, features_dim: int = 128):
        super(CustomObservationExtractor, self).__init__(observation_space, features_dim)
        self._extractor = nn.Sequential(
            nn.Linear(int(np.prod(observation_space.shape)), features_dim),
            nn.ReLU(),
        )

    def forward(self, observations: th.Tensor) -> th.Tensor:
        return self._extractor(observations)

# 第二步：自定义ActorCriticPolicy
class CustomActorCriticPolicy(ActorCriticPolicy):
    def _build_mlp_extractor(self) -> None:
        self.mlp_extractor = CustomObservationExtractor(self.observation_space, features_dim=128)

    def _build_mlp(self, input_dim: int, output_dim: int, architecture: list) -> nn.Module:
        """构造不等规模的actor和critic"""
        modules = []
        for dim in architecture:
            modules.append(nn.Linear(input_dim, dim))
            modules.append(nn.ReLU())
            input_dim = dim
        modules.append(nn.Linear(input_dim, output_dim))
        return nn.Sequential(*modules)

    def _build(self, lr_schedule: Callable[[float], float]) -> None:
        # 区别地自定义Actor和Critic的表示
        self.actor = self._build_mlp(self.features_dim, self.action_space.shape[0], [64])  # 简化的Actor架构
        self.critic = self._build_mlp(self.features_dim, 1, [128, 128])  # 加强的Critic架构
        self.optimizer = th.optim.Adam(self.parameters(), lr=lr_schedule(1))

# 环境实例化
env = gym.make("Pendulum-v1")

# 模型实例化并训练
model = PPO(CustomActorCriticPolicy, env, verbose=1)
model.learn(total_timesteps=10000)
```

### 解释

- **自定义表示**：首先，通过`CustomObservationExtractor`为来自环境的不同形态观察数据抽取初始表示。
- **差异化Actor-Critic**：`CustomActorCriticPolicy`进而区别化定义Actor和Critic的表示结构。这在处理需要复杂估值表示而控制表示较为直接的任务中尤为关键。
- **环境与模型**：实例化环境，之后应用自定义表示实现的PPO算法来指导智能体就环境所定义的连续动作空间任务做决策。
- **学习与实践**：在`PPO`算法环境与自定义表示基础上应用`learn`函数启动智能体的任务策略习得过程，加以实践与探索。

本例子向我们明晰在连续动作空间任务的背景之下，针对Actor和Critic表示加以差异化设计和部署的操作过程与实施思路，展现出应用自定义表示习得复杂任务中所蕴含不对等关系决策表示潜力与策略的实际应用效益。通过这样的自定义Pipeline，研究人员与实践人员得以针对蕴涵任务在表示与表示习得方法上进行深入的微调与创新实践。这强化了针对连续决策制定任务的PPO算法或其他算法的表示效能，为处理高度复杂和差异化表示论域的连续控制问题贡献了重要实践与实验方法。

# Off-Policy Algorithms

在应用如SAC（Soft Actor-Critic）、DDPG（Deep Deterministic Policy Gradient）、TQC（Truncated Quantile Critics）或TD3（Twin Delayed DDPG）这样的离策略（Off-Policy）强化学习算法时，经常面临需要针对Actor（智能体，亦称为`pi`）和Critic（Q函数，评估者，亦称为`qf`）设计差异化的网络架构。这有益于针对具体任务调谐表示能力和决策能力的差异，达成有效决策和精准评价环境和动作结果之间复杂映射的目的。

### 自定义Actor和Critic网络架构

通过为Actor和Critic设置差异化的网络架构，你可以根据任务需求来加以平衡他们的表示学习和决策决策能力。此方法增加智能体表征环境和决策导向的灵活性。以下以SAC算法为例，自定义Actor和Critic网络：

```python
from stable_baselines3 import SAC

policy_kwargs = dict(
    net_arch=dict(
        pi=[64, 64],  # Actor网络架构：两层全连接网络，每层64个单元
        qf=[400, 300]  # Critic网络架构：两层全连接网络，分别为400和300个单元
    )
)

model = SAC(
    "MlpPolicy",
    "Pendulum-v1",
    policy_kwargs=policy_kwargs,
    verbose=1
)

model.learn(total_timesteps=5000)
```

### 深度解析

- **差异化架构设计**：Actor与Critic两个不同的网络架构有其独特意图。Actor需要具备在给定观察情形做决策和输出潜在动作的能力；Critic则重在精确学习和估计这些决策和环境交互产生的长期值。较大规模的Critic网络能帮助处理和习得环境的复杂动态性和估值精度。

- **共享还是独立**：如想共享Actor和Critic的表示，可以设计一单一的网络。在某些应用场景下共享网络能以较少的参数满足习得表示能力，也更为计算有效率。但差异化架构能针对模型学习Actor的行为控制策略和Critic的行为估值策略提供附加灵活性。

- **适用场景**：差异化架构策略一方面提供个性化习得环境表示对计算性和存储开销上存在挑战，但对解空间宽阔、环境动态错综复杂或决策依赖多源信息汇聚的问题定义极具吸引力。

### 小结

差异化Actor和Critic的网络架构是针对连续控制任务中个性化表示习得能力的一种有力设计策略。通过自由选择差异化的表示能力、架构深度和宽度，可以针对任务需求加以有效地习得决策表示，平衡表示丰裕度和决策策略间的权衡，应对强化学习在复杂决策环境中的挑战。此方法能让从业人员针对增强学习议题深入挖掘表示习得和决策估计之间错综的动态互动，有望开创针对具体任务而优化的表示习得新范式。 通过此类范例的学习和实践，您能够拓展自身针对算法的实现，乃至设计层面上针对表示收敛性与适应环境挑战性之间平衡与共生机会的直观理解和操作能力。

本示例将向你介绍如何为SAC（Soft Actor-Critic）算法自定义Actor和Critic网络架构，以便针对连续控制任务比如`HalfCheetah-v2`环境应用差异化的表示习得策略。这在处理连续动作空间和高度复杂的环境交互时特别有益，可帮助算法高效习得决策策略和价值函数。

### 第一步：定义网络架构

差异化地为Actor和Critic网络设计架构：

- Actor（智能体，用于采取动作）：采用较为紧凑的网络架构，例如，这里使用了2个具有64个单元的全连接层。
- Critic（价值函数，即Q函数）：采用较为复杂的网络架构，比如，这里设置了先后具有400和300个单元的2个全连接层。

### 第二步：构造自定义架构

通过`policy_kwargs`字典为SAC算法指定差异化的Actor和Critic网络架构：

```python
from stable_baselines3 import SAC

# 自定义Actor和Critic的网络架构
policy_kwargs = dict(
    net_arch=dict(pi=[64, 64],  # Actor网络架构：两个连续的64单元全连接层
                  qf=[400, 300])  # Critic网络架构：一个400单元接一个300单元的全连接层
)

# 创建模型实例
model = SAC(
    "MlpPolicy", 
    "HalfCheetah-v2", 
    policy_kwargs=policy_kwargs, 
    verbose=1,
    tensorboard_log="./sac_halfcheetah_tensorboard/"
)

# 训练模型
model.learn(total_timesteps=5000, log_interval=10)
```

### 解释

- **差异化设计的考虑**：Actor网络通常被设计得较为紧凑，以增加决策速度并提高灵活性；而Critic网络则往往设计得更为复杂和深入，以便能够学习和建模环境反馈中的复杂非线性动态。

- **`net_arch`字典参数**：通过为`net_arch`参数指明不同的体系结构（这在`sac`的`policy_kwargs`中以`pi`和`qf`关键词设置），允许你在同一智能体内针对Actor和Critic实现差异化的表示习得策略。

- **算法实例化**：使用`SAC`类从SB3库创建了一个算法实例，并传入了自定义Actor和Critic网络架构，这在面对特定连续动作空间问题时尤为关键。

- **模型训练**：之后，调用`.learn()`方法使得智能体在给定环境（此例中为`HalfCheetah-v2`）上学习，旨在优化智能体在环境内连续决策中的表示能力和行动表现。

通过差异化的表示能力设计策略，这种灵活调整连续决策问题上Actor和Critic体系结构的方法可望针对环境习得高效稳健的控制策略，增加了我们针对各种增强学习连续决策任务的处理能力。此外，通过将学习曲线记录到TensorBoard，可以方便追踪和诊断智能体学习过程中的性能表现。这进而为环境的表示学习、动态调参和智能体决策习得的微调调研提供实验借鉴。 